## Step 1: Pre-Preprocessing

In [1]:
%load_ext autoreload
%autoreload 2

# import preprocessing code
from src.preprocess import PreProcessor, df_to_train_set, df_to_XY

Using TensorFlow backend.


In [2]:
# save paths to the available datasets
from typing import NamedTuple, List

class Dataset(NamedTuple):
    """
    Interface for accessing data folders.
    """
    title: str
    preprocessed_folder: str
    raw_folders: List[str]

SAMPLE_DATA = Dataset(
    title = "sample_data",
    preprocessed_folder = "../de-ID_data/preprocessed/sample_data/",
    raw_folders = ["docs/Track1-de-indentification/PHI/"]
)

GOLD_1 = Dataset(
    title = "gold_1",
    preprocessed_folder = "../de-ID_data/preprocessed/gold_1/",
    raw_folders = ["../de-ID_data/raw/training-PHI-Gold-Set1/"]
)

GOLD_FULL = Dataset(
    title = "gold_full",
    preprocessed_folder = "../de-ID_data/preprocessed/gold_full/",
    raw_folders = ["../de-ID_data/raw/training-PHI-Gold-Set1/","../data/raw/training-PHI-Gold-Set2/"]
)

GOLD_TEST = Dataset(
    title = "gold_test",
    preprocessed_folder = "../de-ID_data/preprocessed/gold_test/",
    raw_folders = ["../de-ID_data/raw/testing-PHI-Gold-fixed/"]
)

DATASETS = [SAMPLE_DATA,GOLD_1,GOLD_FULL, GOLD_TEST]

## Step 2: Preprocessing

In [3]:
# pick dataset and define loading boolean
train_data = DATASETS[2]
# train_data = DATASETS[0]
test_data = DATASETS[3]
isLoading = True

In [4]:
# attach data to PreProcessor object.
pp = PreProcessor(train_data.title)
if isLoading:
    X_train,y_train,X_train_words,df_train = pp.get_data(train_data.preprocessed_folder,isLoading = isLoading)
else:
    X_train,y_train,X_train_words,df_train = pp.get_data(train_data.raw_folders,isLoading = isLoading)
print("max length: ",pp.max_len)

Loading preprocessed data...
100%|██████████| 31535/31535 [00:02<00:00, 11805.30it/s]
Preprocessing complete.
max length:  1567


In [9]:
# data exploration
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 627208 entries, 0 to 627207
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  627208 non-null  int64 
 1   docid       627208 non-null  object
 2   sentence    627208 non-null  int64 
 3   token       627138 non-null  object
 4   token_id    627208 non-null  int64 
 5   label       627208 non-null  object
 6   label_id    627208 non-null  int64 
 7   characters  627208 non-null  object
dtypes: int64(4), object(4)
memory usage: 38.3+ MB


In [6]:
# load test set
isLoadingTest = True
if isLoadingTest:
    X_test,y_test,X_test_words,df_test = pp.create_test_set(test_data.preprocessed_folder,isLoadingTest,test_data.title)
else:
    X_test,y_test,X_test_words,df_test = pp.create_test_set(test_data.raw_folders,isLoadingTest,test_data.title)

Loading preprocessed test data...
100%|██████████| 21670/21670 [00:02<00:00, 8751.53it/s]


In [10]:
# test data exploration
df_test.tail()

,Unnamed: 0,docid,sentence,token,token_id,label,label_id,characters,original_token
421834,421834,389-04,17,Team,20785,O,1,"(6839, 6843)",Team
421835,421835,389-04,17,4,9356,O,1,"(6844, 6845)",4
421836,421836,389-04,17,Beeper,23317,O,1,"(6846, 6852)",Beeper
421837,421837,389-04,17,#,8832,O,1,"(6853, 6854)",#
421838,421838,389-04,17,UNK,1,B-PHONE,45,"(6854, 6859)",07736


In [26]:
X_test.shape

(21670, 1536)

In [11]:
# import model stuff
from src.models.baseline import BaselineModel
from src.models.bilstm import BiLSTM
from src.models.bilstm_crf import BiLSTM_CRF
from src.models.transformer import Transformer
from src.models.transformer_crf import Transformer_CRF
from src.models.transformer_bilstm import TransformerBiLSTM
from src.models.bilstm_chars import BiLSTM_Chars
from src.models.bilstm_chars_crf import BiLSTM_Chars_CRF
from pipeline.visualization import sample_output
from pipeline.train import train
from random import randint
from sklearn.utils import shuffle
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
from src.converter import get_label_positions, bio_to_i2d2
import xml.etree.ElementTree as ET
from typing import NamedTuple, List

In [12]:
# check if GPU is available
assert tf.test.is_built_with_cuda()
physical_devices = tf.config.list_physical_devices('GPU') 
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num GPUs:", len(physical_devices)) 
tf.compat.v1.RunOptions(report_tensor_allocations_upon_oom = True)

Num GPUs Available:  1
Num GPUs: 1


report_tensor_allocations_upon_oom: true

In [13]:
# build model
# model = BaselineModel(pp.vocab_size,pp.tag_size,pp.max_len)
# model = BiLSTM(pp.vocab_size,pp.tag_size,pp.max_len)
model = BiLSTM_CRF(pp.vocab_size,pp.tag_size,pp.max_len)
# model = Transformer(pp.vocab_size,pp.tag_size,pp.max_len)
# model = Transformer_CRF(pp.vocab_size, pp.tag_size, pp.max_len)
# model = TransformerBiLSTM(pp.vocab_size, pp.tag_size, pp.max_len)
# model = BiLSTM_Chars(pp.vocab_size, pp.tag_size, pp.max_len,pp.idx2word)
# model = BiLSTM_Chars_CRF(pp.vocab_size, pp.tag_size, pp.max_len,pp.idx2word)

Num GPUs Available:  1


In [14]:
# configure checkpoints and checkpoint manager
checkpoint_dir = 'models/checkpoints/' + train_data.title + '/' + model.title + '/' 
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
checkpoint = tf.train.Checkpoint(model=model)
manager = tf.train.CheckpointManager(checkpoint, checkpoint_dir, max_to_keep=10)

In [15]:
# restore checkpoint
checkpoint.restore(manager.latest_checkpoint)
if manager.latest_checkpoint:
    print("Restored from {}".format(manager.latest_checkpoint))

Restored from models/checkpoints/gold_full/bi-lstm-crf/ckpt-10


In [ ]:
# train
# print("Training ",model.title)
# losses = train(model,X_train,y_train,X_train_words,batch_size = 32, epochs=10, lr = 0.0005, sample_interval=10, manager=manager, pp=pp)

In [19]:
# sample a random train output
sample_output(model,X_train,y_train, pp = pp,rand_idx=None)

Sentence #:  8052
Word            Pred : (True)
He             :O     (O)
has            :O     (O)
an             :O     (O)
occasional     :O     (O)
expiratory     :O     (O)
wheeze         :O     (O)
.              :O     (O)


In [33]:
# sample a random test output
# print(X_test_words[0][:30])
sample_output(model,X_test,y_test, pp = pp,rand_idx=None, words = None)

Sentence #:  18223
input shape:  (1, 1536)
output shape:  (1536,)
Word            Pred : (True)
PE             :O     (O)
:              :O     (O)
BP             :O     (O)
138            :O     (O)
/              :O     (O)
76             :O     (O)
,              :O     (O)
P              :O     (O)
67             :O     (O)
,              :O     (O)
Wt             :O     (O)
199            :O     (O)
lb             :O     (O)
(              :O     (O)
from           :O     (O)
214            :O     (O)
)              :O     (O)
FINGER         :O     (O)
BLOOD          :O     (O)
GLUCOSE        :O     (O)
104            :O     (O)
JVP            :O     (O)
flat           :O     (O)
,              :O     (O)
lungs          :O     (O)
clear          :O     (O)
w              :O     (O)
/              :O     (O)
scant          :O     (O)
crackles       :O     (O)
rrr            :O     (O)
s              :O     (O)
1              :O     (O)
s              :O     (O)
2              :O   

In [34]:
# test model
from pipeline.test import test_to_i2d2

test_to_i2d2(model,df_test, pp, checkpoint, manager)

c:\Users\abdul\Desktop\de-identification
Loading checkpoint...
  0%|          | 0/494 [00:00<?, ?it/s]


TypeError: predict() takes 2 positional arguments but 3 were given